# Score biweekly ML forecasts against re-calibrated ECWMF forecast with RPSS

Goal:

- Score biweekly ML forecasts against re-calibrated ECWMF forecast with RPSS

Requirements:

- [`xskillscore`](https://github.com/xarray-contrib/xskillscore)
- [renku datasets](https://renku-python.readthedocs.io/en/latest/commands.html#module-renku.cli.dataset) / file
    - observations
        - probabilistic:
            - renku dataset: `forecast-like-observations_2020_biweekly_terciled.nc`
    - ML forecasts
        - probabilistic:
            - file: `../submissions/ML_prediction_2020.nc`
    - benchmark:
        - probabilistic:
            - renku dataset: `ecmwf_recalibrated_benchmark_2020_biweekly_terciled.nc` 
        
Output:
- RPSS score spatially averaged from [90N-60S] and averaged over both `lead_time`s and variables

In [5]:
import xarray as xr
import xskillscore as xs
import numpy as np

xr.set_options(keep_attrs=True)
xr.set_options(display_style='text')

cache_path = "../data"

plot = False

/opt/conda/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


# Get categorized

## 2020 observations

In [6]:
# to use retrieve from git lfs
!renku storage pull ../data/forecast-like-observations_2020_biweekly_terciled.nc

In [7]:
obs_p = xr.open_dataset(f'{cache_path}/forecast-like-observations_2020_biweekly_terciled.nc')

obs_p.sizes

/opt/conda/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
/opt/conda/lib/python3.8/site-packages/gribapi/_bindings.cpython-38-x86_64-linux-gnu.so: undefined symbol: codes_bufr_key_is_header
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


Frozen(SortedKeysDict({'category': 3, 'lead_time': 2, 'forecast_time': 53, 'latitude': 121, 'longitude': 240}))

## 2020 ML-based forecasts

In [8]:
!renku storage pull ../submissions/ML_prediction_2020.nc

In [9]:
# submission for your ML model
fct_p = xr.open_dataset(f"../submissions/ML_prediction_2020.nc")
fct_p.sizes

Frozen(SortedKeysDict({'forecast_time': 53, 'latitude': 121, 'lead_time': 2, 'longitude': 240, 'category': 3}))

## 2020 ECMWF recalibrated benchmark

In [10]:
!renku storage pull ../data/ecmwf_recalibrated_benchmark_2020_biweekly_terciled.nc

In [11]:
bench_p = xr.open_dataset('../data/ecmwf_recalibrated_benchmark_2020_biweekly_terciled.nc')
bench_p.sizes

Frozen(SortedKeysDict({'forecast_time': 53, 'category': 3, 'lead_time': 2, 'latitude': 121, 'longitude': 240}))

### checks

In [12]:
from scripts import assert_predictions_2020

In [13]:
assert_predictions_2020(fct_p)
assert_predictions_2020(bench_p)
assert_predictions_2020(obs_p)

# RPS

## RPS ML model

In [14]:
rps_ML = xs.rps(obs_p, fct_p, category_edges=None, dim='forecast_time', input_distributions='p').compute()

In [15]:
if plot:
    for v in rps_ML.data_vars:
        rps_ML[v].plot(robust=True, col='lead_time')

## RPS re-calibrated ECMWF benchmark

In [16]:
rps_bench = xs.rps(obs_p, bench_p, category_edges=None, dim='forecast_time', input_distributions='p').compute()

In [17]:
if plot:
    for v in rps_bench.data_vars:
        rps_bench[v].plot(robust=True, col='lead_time')

## RPSS

In [18]:
rpss = (1 - rps_ML / rps_bench)

In [19]:
if plot:
    for v in rpss.data_vars:
        rpss[v].plot(robust=True, col='lead_time')

# scoring for RPS leaderboard

In [20]:
# check for -inf grid cells
if (rpss==-np.inf).to_array().any():
    (rpss == rpss.min()).sum()

    # dirty fix
    rpss = rpss.clip(-1, 1)

In [21]:
# what to do with requested grid cells where NaN is submitted? also penalize

In [22]:
mask = xr.ones_like(rpss.isel(lead_time=0, drop=True)).reset_coords(drop=True).t2m
boundary_tropics = 30
mask = xr.concat([mask.where(mask.latitude > boundary_tropics),
                mask.where(np.abs(mask.latitude) <= boundary_tropics),
                mask.where((mask.latitude < -boundary_tropics) & (mask.latitude > -60))],'area')
mask = mask.assign_coords(area=['northern_extratropics', 'tropics', 'southern_extratropics'])
mask.name = 'area'

mask = mask.where(rpss.t2m.isel(lead_time=0, drop=True).notnull())

In [23]:
# weighted area mean
weights = np.cos(np.deg2rad(np.abs(mask.latitude)))
scores = (rpss*mask).weighted(weights).mean('latitude').mean('longitude')
pd_scores = scores.reset_coords(drop=True).to_dataframe().unstack(0).round(2)

In [24]:
# final score
scores = rpss.weighted(weights).mean('latitude').mean('longitude')
# spatially weighted score averaged over lead_times and variables to one single value

In [25]:
# score transfered to leaderboard
scores = scores.to_array().mean().reset_coords(drop=True)
print(scores)

<xarray.DataArray ()>
array(-0.00725325)
